* Spearmint for analogy reasoning
* Gaussian LDA
* Evaluate word analogy reasoning
* evalutate topic models
* find background noise
* find word pairs

# Setup

In [1]:
%matplotlib notebook

import itertools
import logging
from functools import partial

import gensim
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pnd
from sklearn.cluster import *
from sklearn.decomposition import PCA, RandomizedPCA
from sklearn.manifold import TSNE

from knub.thesis.util import *
matplotlib.style.use('ggplot')

In [2]:
from IPython.core.display import HTML
HTML("""
<style>
div.text_cell_render p, div.text_cell_render ul, table.dataframe {
font-size:1.3em;
line-height:1.1em;
}
</style>
""")

# Evaluating strength of raw models

## Topic Models → Topic Coherence, Concept Categorization

Evaluated using Palmetto tool from **Exploring the Space of Topic Coherence Measures** paper
Values still seem low compared to example values from the paper

In [3]:
df_tc_results = pnd.DataFrame([
        ("topic.full.alpha-1-100.256-400.model", 0.469500859375, 0.00617111859067, 0.6463414634146342),
        ("topic.16-400.model", 0.43805875, 0.00390183951094, 0.5975609756097561),
        ("topic.256-1000.model", 0.473455351563, 0.00635883046394, 0.5853658536585366),
        ("topic.64-400.model", 0.45327734375, 0.00385141007263, 0.6341463414634146),
        ("topic.256-400.model", 0.46836359375, 0.00599032492068, 0.5731707317073171),
        ("topic.full.fixed-vocabulary.alpha-1-100.256-400.model", 0.468437070312, 0.00562772603243, 0.5975609756097561),
        ("topic.full.256-400.model", 0.472498945313, 0.00624853749772, 0.5975609756097561),
        ("topic.256-600.model", 0.478640273437, 0.00685787139094, 0.5609756097560975)
    ], 
    columns=["Topic model parameters", "TC_mean", "TC_var", "CC_purity"])

In [4]:
df_tc_results.sort_values(by="TC_mean", ascending=False)

,Topic model parameters,TC_mean,TC_var,CC_purity
7,topic.256-600.model,0.478640,0.006858,0.560976
2,topic.256-1000.model,0.473455,0.006359,0.585366
6,topic.full.256-400.model,0.472499,0.006249,0.597561
0,topic.full.alpha-1-100.256-400.model,0.469501,0.006171,0.646341
5,topic.full.fixed-vocabulary.alpha-1-100.256-40...,0.468437,0.005628,0.597561
4,topic.256-400.model,0.468364,0.005990,0.573171
3,topic.64-400.model,0.453277,0.003851,0.634146
1,topic.16-400.model,0.438059,0.003902,0.597561


In [5]:
df_tc_results.sort_values(by="CC_purity", ascending=False)

,Topic model parameters,TC_mean,TC_var,CC_purity
0,topic.full.alpha-1-100.256-400.model,0.469501,0.006171,0.646341
3,topic.64-400.model,0.453277,0.003851,0.634146
1,topic.16-400.model,0.438059,0.003902,0.597561
5,topic.full.fixed-vocabulary.alpha-1-100.256-40...,0.468437,0.005628,0.597561
6,topic.full.256-400.model,0.472499,0.006249,0.597561
2,topic.256-1000.model,0.473455,0.006359,0.585366
4,topic.256-400.model,0.468364,0.005990,0.573171
7,topic.256-600.model,0.478640,0.006858,0.560976


In [6]:
df_tc_results.sort_values(by="TC_var", ascending=False)

,Topic model parameters,TC_mean,TC_var,CC_purity
7,topic.256-600.model,0.478640,0.006858,0.560976
2,topic.256-1000.model,0.473455,0.006359,0.585366
6,topic.full.256-400.model,0.472499,0.006249,0.597561
0,topic.full.alpha-1-100.256-400.model,0.469501,0.006171,0.646341
4,topic.256-400.model,0.468364,0.005990,0.573171
5,topic.full.fixed-vocabulary.alpha-1-100.256-40...,0.468437,0.005628,0.597561
1,topic.16-400.model,0.438059,0.003902,0.597561
3,topic.64-400.model,0.453277,0.003851,0.634146


## Word Embeddings → Analogy Reasoning

Using the question word data set (~19k questions) from **Efficient Estimation of Word Representations in Vector Space** (word2vec).
Also, currently running Spearmint optimization to look for better embeddings.

In [7]:
pnd.set_option("display.max_colwidth", 100)

In [8]:
df_tc_results = pnd.DataFrame([
        ("embedding.skip-gram.size-200.window-5.negative-5.model", 0.481221858371),
        ("embedding.cbow.size-200.window-5.model", 0.416547277937),
        ("embedding.google.size-300", 0.735878018829),
    ], 
    columns=["Word Embeddings", "Analogy_Reasoning"])

df_tc_results.sort_values(by="Analogy_Reasoning", ascending=False)

,Word Embeddings,Analogy_Reasoning
2,embedding.google.size-300,0.735878
0,embedding.skip-gram.size-200.window-5.negative-5.model,0.481222
1,embedding.cbow.size-200.window-5.model,0.416547


# Preprocessing

In [9]:
MODEL = "../models/topic-models/topic.full.alpha-1-100.256-400.model"

In [10]:
print "Load vectors"
vectors = load_skip_gram()
model = TopicModelLoader(MODEL, vectors)
print "Load topic probs"
df_topic_probs_full = model.load_topic_probs()
print "Load topics"
df_topics = model.load_topics()
print "Load topic similars"
df_topic_similars = model.load_all_topic_similars()

Load vectors
Load topic probs
Load topics
Load topic similars


# Topic Probs Analysis

In [11]:
df_topic_probs = df_topic_probs_full[df_topic_probs_full["word"].apply(lambda w: w in model.topic_words)].copy()
df_topic_probs["stddev"] = df_topic_probs[model.prob_columns].std(axis=1)

In [12]:
df_topic_probs.head(3)[model.prob_columns].mean(axis=1)*256

1    1.0
2    1.0
5    1.0
dtype: float64

### Highest std. dev.

In [13]:
df_topic_probs.sort_values(by="stddev", ascending=False).head(10)[["word", "stddev"]]

,word,stddev
97299,gmina,0.062500
15107,cerambycidae,0.062500
5051,forewings,0.062341
12297,rabbi,0.061799
5040,moth,0.061736
26868,nascar,0.061622
12185,nigerian,0.061617
97298,voivodeship,0.061585
29709,kuala,0.061550
30707,nhl,0.061477


### Lowest std. dev.

In [14]:
df_topic_probs.sort_values(by="stddev", ascending=True).head(10)[["word", "stddev"]]

,word,stddev
169,one,0.002358
188,also,0.002516
302,two,0.002998
430,first,0.003318
354,known,0.003707
483,time,0.004129
680,part,0.004133
412,later,0.004259
2844,however,0.004622
2167,early,0.004699


# Correlation TM similarity and WE similarity

### Ten most similar words for each top-10-topic word

 Topic model similarity evaluated using different probability distribution similarity measures (evaluated on the normalized word-topic distributions):
 
 * [Jensen-Shannon divergence](https://en.wikipedia.org/wiki/Jensen%E2%80%93Shannon_divergence)
 * Hellinger distance
 * Bhattacharyya coefficient

In [15]:
df_topic_similars.head()

,word,similar_word,tm_sim_max,tm_sim_sum,tm_sim_bhattacharyya,tm_sim_hellinger,tm_sim_jensen-shannon,we_sim
0,military,staffs,0.912480,0.558798,0.766800,0.517092,0.118427,0.318163
1,military,army,0.912530,0.562705,0.768054,0.518392,0.119736,0.608158
2,military,forces,0.916243,0.572355,0.768518,0.518874,0.133331,0.571750
3,military,peacetime,0.916459,0.580579,0.786892,0.538363,0.141890,0.615062
4,military,defence,0.919340,0.603073,0.789961,0.541700,0.166720,0.493413


### Correlation between TM and WE similarity

In [16]:
model.sim_functions = ["max", "sum", "bhattacharyya", "hellinger", "jensen-shannon"]

sim_corrs_spearman = []
sim_corrs_pearson = []
for sim_function in model.sim_functions:
    corr_spearman = df_topic_similars[["tm_sim_%s" % sim_function, "we_sim"]].corr("spearman").ix[0,1]
    corr_pearson = df_topic_similars[["tm_sim_%s" % sim_function, "we_sim"]].corr("pearson").ix[0,1]
    sim_corrs_spearman.append(corr_spearman)
    sim_corrs_pearson.append(corr_pearson)

df_tmp = pnd.DataFrame(model.sim_functions, columns=["sim_function"])
df_tmp["sim_corr_spearman"] = sim_corrs_spearman
df_tmp["sim_corr_pearson"] = sim_corrs_pearson
df_tmp

,sim_function,sim_corr_spearman,sim_corr_pearson
0,max,-0.076134,-0.043531
1,sum,0.485480,0.467992
2,bhattacharyya,0.417126,0.424787
3,hellinger,0.417126,0.412042
4,jensen-shannon,-0.349793,-0.350979


**Note: Similar results Google vectors**

### Distribution of TM similarity

In [17]:
plt.figure()
df_topic_similars["tm_sim_jensen-shannon"].hist(bins=100)

<IPython.core.display.Javascript object>

### Distribution of WE similarity

In [18]:
plt.figure()
df_topic_similars["we_sim"].hist(bins=50)

<IPython.core.display.Javascript object>

In [19]:
we_percentile = df_topic_similars["we_sim"].quantile(q=.30)
we_percentile

0.20486269730143181

In [20]:
df_tmp = df_topic_probs[["word", "stddev"]]
df_tmp.columns = ["w", "stddev"]
df_result = df_topic_similars.merge(df_tmp, left_on="similar_word", right_on="w")
del df_result["w"]
word_prob_quantile = df_result["stddev"].quantile(0.8)

### High TM similarity, low WE similarity

In [21]:
df_large_sim_diff = df_result[(df_result["we_sim"] < 0.4) & (df_result["stddev"] > 0.025)]
df_large_sim_diff.iloc[np.random.permutation(len(df_large_sim_diff))]

,word,similar_word,tm_sim_max,tm_sim_sum,tm_sim_bhattacharyya,tm_sim_hellinger,tm_sim_jensen-shannon,we_sim,stddev
789,households,females,0.891974,0.795478,0.884133,0.659607,0.088043,0.250761,0.030429
1441,animals,wolf,0.940993,0.684543,0.820232,0.576009,0.094401,0.251635,0.031948
2060,manila,filipino,0.969272,0.954975,0.976968,0.848236,0.017948,0.369056,0.059142
328,columbia,provincial,0.909951,0.518349,0.699663,0.451970,0.238158,0.165495,0.025065
1600,constituency,liberal,0.888191,0.662749,0.801406,0.554361,0.151110,0.315658,0.027806
2051,refer,surname,0.969727,0.919300,0.958234,0.795633,0.030982,0.301821,0.058946
2045,awards,miss,0.945258,0.717227,0.832901,0.591223,0.164896,0.228340,0.040412
2015,vietnam,kim,0.948447,0.809883,0.904104,0.690328,0.070119,0.197542,0.050290
2018,cambodia,korean,0.962699,0.899267,0.944429,0.764264,0.031369,0.223107,0.054569
466,austria,german,0.839714,0.636726,0.807103,0.560800,0.171045,0.383683,0.028347


### High TM similarity, high WE similarity

In [22]:
df_small_sim_diff = df_result[(df_result["we_sim"] > 0.8) & (df_result["stddev"] > 0.025)]
df_small_sim_diff.iloc[np.random.permutation(len(df_small_sim_diff))]

,word,similar_word,tm_sim_max,tm_sim_sum,tm_sim_bhattacharyya,tm_sim_hellinger,tm_sim_jensen-shannon,we_sim,stddev
2023,drugs,drug,0.961754,0.853085,0.964254,0.810935,0.133683,0.839987,0.029113
1599,conservative,liberal,0.904876,0.738297,0.892804,0.672592,0.164209,0.886612,0.027806
1520,locomotives,locomotive,0.985610,0.978550,0.994476,0.925679,0.018198,0.844325,0.059194
1594,liberal,conservative,0.904876,0.746874,0.888088,0.665467,0.141828,0.886612,0.028347
635,bass,guitar,0.908315,0.687827,0.840264,0.600330,0.119666,0.915111,0.030964
638,bishop,archbishop,0.904739,0.834983,0.931139,0.737585,0.044528,0.898991,0.038320
1527,buenos,aires,0.995521,0.994942,0.999415,0.975809,0.006205,0.899445,0.060902
1545,francisco,diego,0.934812,0.747282,0.866766,0.634988,0.111047,0.921034,0.032479
2038,award,awards,0.866506,0.562245,0.735712,0.485910,0.152539,0.833322,0.037197
1965,jacques,pierre,0.953684,0.783048,0.868368,0.637188,0.113166,0.889388,0.042466


# Findings

* syntatic variations play a bigger role in WE models, example:

  **(development, developed)**: TM-sim: 0.960519 WE-SIM: 0.360895
  
  **(composed, composers)** TM-SIM: 0.973376 WE-SIM: 0.329483
  
  **(works, working)** TM-SIM: 0.969470 WE-SIM: 0.274090
* topic models are better at capturing loose relationships, such as:

  **(war, commander)** TM-SIM: 0.922352 WE-SIM: 0.187498
  
  **(living, households)** TM-SIM: 0.983162 WE-SIM: 0.207906
  
  **(county, rural)** TM-SIM: 0.882099 WE-SIM: 0.257984
  

# Concept categorization in TM and WE

Roughly the same results after using the same algorithm for both systems

In [23]:
def get_embedding_from_word_embedding(word):
    try:
        return vectors[word]
    except:
        return vectors["this"]

columns = [str(i) for i in range(256)]
def get_embedding_from_topics(word):
    df_row = df_probs[df_probs["word"] == word]
    assert len(df_row) == 1, "not exactly one row found: " + word + " " + len(df_row)
    return df_row[columns].iloc[0,:].tolist()

def get_df_concept(embedding_function):
    df_concept = pnd.read_csv(
        "/home/knub/Repositories/master-thesis/data/concept-categorization/battig_concept-categorization.tsv",
        sep="\t",
        header=None)
    df_concept.columns = ["word", "concept"]
    df_concept["embeddings"] = df_concept["word"].apply(embedding_function)
    return df_concept

df_we_concept = get_df_concept(get_embedding_from_word_embedding)
df_tm_concept = get_df_concept(get_embedding_from_topics)
df_tm_concept.head(2)

NameError: global name 'df_probs' is not defined

In [ ]:
len(df_tm_concept.ix[0,"embeddings"])

In [ ]:
from sklearn import metrics

# http://stats.stackexchange.com/questions/95731/how-to-calculate-purity
def single_cluster_purity(df_param):
    return df_param["concept"].value_counts().max()

def calculate_purity(df_param):
    purity = float(sum([single_cluster_purity(df_cluster_group)
                        for _, df_cluster_group
                        in df_param.groupby("cluster_id")])) / len(df_param)
    return purity


def evaluate_clustering_algorithm(df_param, clustering):
    X = np.array(df_param["embeddings"].tolist())
    X_sim = metrics.pairwise.pairwise_distances(X, metric="cosine")
    # sim or not sim? PCA or not PCA?
    clusters = clustering.fit_predict(pca(X_sim, 10))
    df_param["cluster_id"] = clusters
    return calculate_purity(df_param)

In [ ]:
for df_concept in [df_we_concept, df_tm_concept]:
    print "-" * 100
    for clustering in [KMeans(n_clusters=10, init="k-means++", n_jobs=1),
                       AgglomerativeClustering(n_clusters=10, linkage="ward"),
                       AgglomerativeClustering(n_clusters=10, linkage="complete"),
                       AgglomerativeClustering(n_clusters=10, linkage="average"),
                       AffinityPropagation(damping=0.5),
                       AffinityPropagation(damping=0.6),
                       AffinityPropagation(damping=0.7),
                       AffinityPropagation(damping=0.8),
                       AffinityPropagation(damping=0.9),
                   SpectralClustering(n_clusters=3)]:
        print clustering.__class__.__name__
        print evaluate_clustering_algorithm(df_concept, clustering)

# Word Similarity

## Similarity

In [ ]:
def word_similarity(f):
    try:
        df_sim = pnd.read_csv(MODEL + f, sep="\t")
        df_sim["embedding-sim"] = df_sim[["word1", "word2"]].apply(
            lambda x: get_similarity(x["word1"], x["word2"], vectors), axis=1)
        df_sim["comb-sim"] = df_sim[["topic-sim", "embedding-sim"]].apply(
            lambda x: (5 * x["embedding-sim"] + x["topic-sim"]) / 6,
            axis=1)
        
        topic_corr     = df_sim[["human-sim", "topic-sim"]].corr("spearman").ix[0,1]
        embedding_corr = df_sim[["human-sim", "embedding-sim"]].corr("spearman").ix[0, 1]
        
        topic_corr_pearson  = df_sim[["human-sim", "topic-sim"]].corr("pearson").ix[0,1]
        embedding_corr_pearson = df_sim[["human-sim", "embedding-sim"]].corr("pearson").ix[0, 1]
        
        comb_corr      = df_sim[["human-sim", "comb-sim"]].corr("spearman").ix[0, 1]
        return pnd.DataFrame([[topic_corr, embedding_corr, topic_corr_pearson, embedding_corr_pearson]],
                             columns=["topic_corr", "embedding_corr", "topic_corr_pearson", "embedding_corr_pearson"],
                             index=[f])
    except:
        return None

pnd.concat([word_similarity(".wordsim353-sim"),
            word_similarity(".wordsim353-rel"),
            word_similarity(".wordsim353-all"),
            word_similarity(".scws"),])